# 30Q GHZ state trial on Manhattan

This notebook requires: `Qiskit >= 0.24`, and of course `mthree >= 0.4.1`.  The latter must be installed from source at the moment.

This will attempt to execute a 30Q GHZ state using Manhattan.  This is interesting as forming the full mitigation matrix would require $2^{60}$ elements.


In [1]:
import time
import numpy as np
from qiskit import *
from qiskit.ignis.mitigation import expectation_value

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(group='deployed')
backend = provider.backend.ibmq_manhattan

In [3]:
import mthree
mthree.about()

# Matrix-free Measurement Mitigation (M3) version 0.4.1.dev1+1fabf99
# (C) Copyright IBM Quantum, 2021
# Paul Nation, Hwajung Kang, and Jay Gambetta


In [4]:
layers = [[4], 
          [5], 
          [6,11], 
          [7,17,3], 
          [8,18,2], 
          [12,19,1], 
          [21,25,0],
          [22,33,10],
          [23,32,13],
          [26,31,14],
          [37,39,15],
             [45,24]]

qubits = sum(layers, [])
len(qubits)

30

### Build 30 qubit GHZ circuit

In [5]:
qc = QuantumCircuit(65, len(qubits))
qc.h(4)
#branch 1
qc.cx(4,5)
qc.cx(5,6)
qc.cx(6,7)
qc.cx(7,8)
qc.cx(8,12)
qc.cx(12,21)
qc.cx(21,22)
qc.cx(22,23)
qc.cx(23,26)
qc.cx(26,37)
#branch 2
qc.cx(4,11)
qc.cx(11,17)
qc.cx(17,18)
qc.cx(18,19)
qc.cx(19,25)
qc.cx(25,33)
qc.cx(33,32)
qc.cx(32,31)
qc.cx(31,39)
qc.cx(39,45)
#branch 3
qc.cx(4,3)
qc.cx(3,2)
qc.cx(2,1)
qc.cx(1,0)
qc.cx(0,10)
qc.cx(10,13)
qc.cx(13,14)
qc.cx(14,15)
qc.cx(15,24)
qc.measure(qubits, range(len(qubits)))

### Instantiate M3 Instance

In [6]:
mit = mthree.M3Mitigation(backend)

### Calibration

In [7]:
mit.tensored_cals_from_system(qubits)

### Execute the circuit on the backend and compute the expectation value

In [11]:
ghz_job = execute(qc, backend, shots=8192, optimization_level=1)

In [12]:
real_counts = ghz_job.result().get_counts()
len(real_counts)

3874

In [13]:
expectation_value(real_counts)[0]

0.0849609375

### Measurement mitigation by M3 and compute the expectation value

In [14]:
quasi, details = mit.apply_correction(real_counts, qubits=qubits, details=True)
details

{'method': 'direct',
 'time': 0.7396790981292725,
 'dimension': 3874,
 'col_norms': array([0.9033827 , 0.42526317, 0.71993441, ..., 0.61167295, 0.52939782,
        0.78016068])}

In [15]:
expectation_value(quasi)[0]

0.5027474694623298